GPU Setting -> GPU 1번 사용

In [ ]:
import tensorflow as tf
cpus = tf.config.experimental.list_physical_devices('CPU')
if cpus:
    try:
        tf.config.experimental.set_visible_devices(cpus[0], 'CPU')
    except RuntimeError as e:
        print(e)

In [ ]:
visible_devices = tf.config.get_visible_devices()
print(visible_devices)

Transformer 모델의 구성 -> Transformer_Implement.ipynb

In [ ]:
import import_ipynb
import Transformer_Implement as ti

import re
import os
import numpy as np
from pathlib import Path
from gensim.models import Word2Vec, KeyedVectors

경로 설정

In [ ]:
BASE_DIR = "/data/ksb/TestDir"
DATA_BASE_DIR = os.path.join(BASE_DIR, 'sample_articles')

PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")
MINI_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Mini-Preprocessed-Data")

SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Summary-Preprocessed-Data")
MINI_SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Mini-Summary-Preprocessed-Data")

MODEL_BASE_DIR = os.path.join(Path(os.getcwd()).parent, 'Word-Embedding-Model')
word2vec_model_path = os.path.join(MODEL_BASE_DIR, 'word2vec-256.model')
wv_model_path = os.path.join(MODEL_BASE_DIR, 'word2vec-256.wordvectors')

In [ ]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def del_folder(path):
    from shutil import rmtree
    try:
        rmtree(path)
    except Exception as e:
        print(e)
        pass

In [ ]:
def get_switched_root_dir(path, save_root_dir):
    path_token = path.split(os.sep)
    path_token[4] = save_root_dir.split(os.sep)[4]
    save_dir_path = os.sep.join(path_token[:6])
    
    return save_dir_path

In [ ]:
class ArticleCopier:
    def __init__(self, path):
        with open(path, 'r', encoding='utf-8') as f:
            self.lines = f.readlines()

    def copy_article(self, save_dir):
        mkdir_p(save_dir)
        article_name = str(len(os.listdir(save_dir)))+".txt"

        with open(os.path.join(save_dir, article_name), 'w', encoding='utf-8') as new_f:
            new_f.writelines(self.lines)

def load_proc_data(summary_path):

    media_list = sorted(os.listdir(summary_path))
        
    for media_name in media_list:
    
        media_path = os.path.join(summary_path, media_name)
        article_list = os.listdir(media_path)
            
        article_path_list = [os.path.join(media_path, article_name) for article_name in article_list]
        
        origin_root = PREPROCESSED_PATH
        new_root = MINI_PREPROCESSED_PATH
        
        origin_proc_path_list = [os.path.join(get_switched_root_dir(path, origin_root), path.split(os.sep)[-1]) for path in article_path_list]
        new_proc_path_list = [get_switched_root_dir(path, new_root) for path in article_path_list]

        for origin_path, new_path in zip(origin_proc_path_list, new_proc_path_list):
            article = ArticleCopier(origin_path)
            article.copy_article(new_path)
        map(lambda article : article.copy_article, new_proc_path_list)

In [ ]:
#del_folder(MINI_PREPROCESSED_PATH)
#MINI_PREPROCESSED_PATH

In [ ]:
#load_proc_data(MINI_SUMMARY_PREPROCESSED_PATH)

말뭉치와 함께 임베딩 모델 불러오기

In [ ]:
word2vec = Word2Vec.load(word2vec_model_path)
wv = KeyedVectors.load(wv_model_path, mmap='r')

In [ ]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s

In [ ]:
def get_embedded_list(filepath, wv):
    media_list = os.listdir(filepath)
    embedding_vec_list = []
    for media_name in media_list:
    
        media_path = os.path.join(filepath, media_name)
        article_list = os.listdir(media_path)
            
        for article_name in article_list:
                
            reader = RawTextReader(os.path.join(media_path, article_name)) 
            content = list(filter(None, reader))
                
            vec = np.array([wv.word_vec(token) for sent in content for token in sent.split() if token in wv])
            embedding_vec_list.append(vec)

    return embedding_vec_list

In [ ]:
D_MODEL = 256
LAYER_NUM = 6
NUM_HEADS = 8
DFF = 512
VOCAB_SIZE = wv.vectors.shape[0]

BATCH_SIZE = 64
BUFFER_SIZE = 20000

WARMUP_STEPS = 50
EPOCHS = 70

전처리 된 59,604개의 기사 본문 데이터를 임베딩 벡터로 변환한다

각 기사마다 토큰 개수에 따라 (None, 256)의 Matrix를 가진다.

각 기사의 토큰에 대해 임베딩을 수행한 결과 리스트를 `train_embedded_list`, `target_embedded_list`로 저장한다.

In [ ]:
train_embedded_list = get_embedded_list(MINI_PREPROCESSED_PATH, wv)
target_embedded_list = get_embedded_list(MINI_SUMMARY_PREPROCESSED_PATH, wv)

In [ ]:
get_max_length = lambda x : np.max([len(x[idx]) for idx in range(len(x))])
MAX_LEN = get_max_length(train_embedded_list)
MAX_LEN

In [ ]:
target_max_len = get_max_length(target_embedded_list)
MAX_LEN = target_max_len if target_max_len > MAX_LEN else MAX_LEN
MAX_LEN

각 문장의 토큰을 임베딩 벡터로 변환한다.  

각 인코딩된 값을 임베딩 벡터 값으로 대체한 후에는 (59604, `max_token`, 256)의 크기를 가지는 Tensor가 된다.  
- `max_token`은 각 기사 본문의 토큰 수 중 가장 큰 값을 의미한다.

이의 결과로 (59604, 4392, 256)의 dimension을 가진다.

In [ ]:
train_embedded_matrix = tf.keras.preprocessing.sequence.pad_sequences(train_embedded_list, maxlen=40, padding='post', dtype=np.float32)
target_embedded_matrix = tf.keras.preprocessing.sequence.pad_sequences(target_embedded_list, maxlen=40, padding='post', dtype=np.float32)

print("Train Embedded Matrix shape : {train} \nTargetEmbedded Matrix shape : {target}"\
      .format(train=train_embedded_matrix.shape,
             target=target_embedded_matrix.shape))

`dataset` 구조  
```
({'encoder_inputs' : [[[...]]], 'decoder_inputs' : [[[...]]] }, { 'Output' : [[[...]]] }) 의 튜플이 59604개 있다.
```


In [ ]:
decoder_inputs = target_embedded_matrix
decoder_inputs[:, -1, :] = 0.0

outputs = target_embedded_matrix
outputs[:, 1, :] = 0.0


dataset = tf.data.Dataset.from_tensor_slices((
    {
        'encoder_inputs': train_embedded_matrix, # Encoder Input
        'decoder_inputs': decoder_inputs # Decoder Input
    },
    {
        # Decoder Output, Remove <SOS>
        'Output': outputs 
    },
))

In [ ]:
for dict_1, dict_2 in dataset.as_numpy_iterator():
    print("encoder input : {enc}, decoder input : {dec}".format(enc= dict_1['encoder_inputs'].shape,
                                                               dec=dict_1['decoder_inputs'].shape))
    print("output shape : {}".format(dict_2['Output'].shape))

데이터의 59604를 `BATCH_SIZE(=64)`에 따라 분할하며, 
dataset의 크기는 아래와 같다.

In [ ]:
# 데이터셋을 메모리 또는 로컬 저장소에 캐시, 각 epoch 동안 실행되는 일부 작업(파일 열기 및 데이터 읽기 등)이 저장됨
dataset = dataset.cache() 
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for dict_1, dict_2 in dataset.as_numpy_iterator():    
    print("encoder input : {enc}, decoder input : {dec}".format(enc= dict_1['encoder_inputs'].shape,
                                                               dec=dict_1['decoder_inputs'].shape))
    print("output shape : {}".format(dict_2['Output'].shape))

학습률 Learning Rate 조정

In [ ]:
lrate_scheduler = ti.LearningRate(d_model=D_MODEL, warmup_steps=WARMUP_STEPS)

Optimizer 정의

In [ ]:
beta_1 = 0.9  
beta_2 = 0.98
epsilon = 10 ** -9

optimizer = tf.keras.optimizers.Adam(lrate_scheduler, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

Transformer 모델 정의

In [ ]:
with tf.device('/CPU:0'):
    model = ti.Transformer(
        vocab_size=VOCAB_SIZE,
        layer_num=LAYER_NUM,
        dff=DFF,
        d_model=D_MODEL,
        num_heads=NUM_HEADS).get_transformer()

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.00005, verbose=1)

model.fit(dataset, batch_size=16, epochs=EPOCHS, shuffle=True, callbacks=[reduce_lr])